In [1]:
!pip install fastBPE sacremoses subword_nmt 

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 36.6 MB/s eta 0:00:00
  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp310-cp310-linux_x86_64.whl size=806540 sha256=5ef63d0ae5eba7aa4615412f576d7675361dd30cedc928b97aaa7a799c5280de
  Stored in directory: /root/.cache/pip/wheels/13/5d/b9/4b8897941ebc9e8c6cc3f3ffd3ea5115731754269205098754
Successfully built fastBPE


In [2]:
!pip install scikit-learn 
!pip install scipy

In [3]:

pip install hydra-core bitarray sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch

# List available models
torch.hub.list('pytorch/fairseq')  # [..., 'transformer.wmt16.en-de', ... ]

# Load a transformer trained on WMT'16 En-De
# Note: WMT'19 models use fastBPE instead of subword_nmt, see instructions below
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt14.en-fr',
                       tokenizer='moses', bpe='subword_nmt')
en2de.eval()  # disable dropout

# The underlying model is available under the *models* attribute
# assert isinstance(en2de.models[0], fairseq.models.transformer.TransformerModel)

# Move model to GPU for faster translation
en2de.cuda()

# Translate a sentence
en2de.translate('Hello world!')
# 'Hallo Welt!'

# Batched translation
en2de.translate(['Hello world!', 'The cat sat on the mat.'])
# ['Hallo Welt!', 'Die Katze saß auf der Matte.']

Downloading: "https://github.com/pytorch/fairseq/zipball/main" to /root/.cache/torch/hub/main.zip
/root/.cache/torch/hub/pytorch_fairseq_main/fairseq/tasks/multires_hubert_pretraining.py:154: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  dictionaries = [ (Dictionary.load(f"{label_dir}/dict.{label}.txt") if label is not "" else None ) for label in self.cfg.labels]
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/dist.py:261: UserWarning: Unknown distribution option: 'test_suite'
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:416: UserWarning: The detected CUDA version (12.2) has a minor version mismatch with the version that was used to compile PyTorch (12.1). Most likely this shouldn't be a problem.
  warnings.warn(CUDA_MISMATCH_WARN.format(cuda_str_version, torch.version.cuda))
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:426: UserWarning: There are no x86_64-linux-gnu-g++ version bounds def

[1/1] Cythonizing fairseq/data/data_utils_fast.pyx


Emitting ninja build file /root/.cache/torch/hub/pytorch_fairseq_main/build/temp.linux-x86_64-cpython-310/build.ninja...
Compiling objects...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/1] Cythonizing fairseq/data/token_block_utils_fast.pyx


Emitting ninja build file /root/.cache/torch/hub/pytorch_fairseq_main/build/temp.linux-x86_64-cpython-310/build.ninja...
Compiling objects...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Emitting ninja build file /root/.cache/torch/hub/pytorch_fairseq_main/build/temp.linux-x86_64-cpython-310/build.ninja...
Compiling objects...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Emitting ninja build file /root/.cache/torch/hub/pytorch_fairseq_main/build/temp.linux-x86_64-cpython-310/build.ninja...
Compiling objects...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Emitting ninja build file /root/.cache/torch/hub/pytorch_fairseq_main/build/temp.linux-x86_64-cpython-310/build.ninja...
Compiling objects...
Allowing ninja to set a default number of workers... (overridable by setting the environme

['Bonjour à tous !', 'Le chat était assis sur le tapis.']

In [5]:
param_dict = {}
for name, param in en2de.named_parameters():
    if 'weight' in name and 'layernorm' not in name.lower():
        param_dict[name] = param.detach().cpu().numpy()

In [6]:
import numpy as np
import torch

In [7]:
def class_uniform_pruning(param_dict, prune_ratio):
    mask_dict = {}
    for name, weights in param_dict.items():
        threshold = np.percentile(np.abs(weights), prune_ratio * 100)
        mask = np.abs(weights) > threshold
        mask_dict[name] = mask
    return mask_dict

In [8]:
def class_blind_pruning(param_dict, prune_ratio):
    all_weights = np.concatenate([w.flatten() for w in param_dict.values()])
    threshold = np.percentile(np.abs(all_weights), prune_ratio * 100)

    mask_dict = {}
    for name, weights in param_dict.items():
        mask = np.abs(weights) > threshold
        mask_dict[name] = mask
    return mask_dict

In [9]:
def class_distribution_pruning(param_dict, prune_ratio):
    # Search for lambda such that total pruned weights ≈ x%
    def compute_total_pruned(lambda_val):
        total = 0
        pruned = 0
        for weights in param_dict.values():
            sigma = np.std(weights)
            threshold = lambda_val * sigma
            pruned += np.sum(np.abs(weights) < threshold)
            total += weights.size
        return pruned / total

    # Binary search for lambda
    low, high = 0.0, 10.0
    for _ in range(20):
        mid = (low + high) / 2
        ratio = compute_total_pruned(mid)
        if ratio < prune_ratio:
            low = mid
        else:
            high = mid
    lambda_opt = (low + high) / 2

    # Generate mask using the found lambda
    mask_dict = {}
    for name, weights in param_dict.items():
        sigma = np.std(weights)
        threshold = lambda_opt * sigma
        mask = np.abs(weights) > threshold
        mask_dict[name] = mask
    return mask_dict

In [10]:
def apply_masks_to_model(model, mask_dict):
    for name, param in model.named_parameters():
        if name in mask_dict:
            mask = torch.tensor(mask_dict[name], dtype=param.dtype, device=param.device)
            with torch.no_grad():
                param.mul_(mask)  # In-place pruning
    return model


In [11]:
# mask_dict = class_blind_pruning(param_dict,0.1)
# pruned_model = apply_masks_to_model(en2de, mask_dict)
# torch.save(pruned_model.state_dict(), 'pruned_model_weights.pt')

In [12]:
# mask_dict_1 = class_uniform_pruning(param_dict,0.1)
# pruned_model_blind = apply_masks_to_model(en2de, mask_dict_1)
# torch.save(pruned_model_blind.state_dict(), 'pruned_model_blind_weights.pt')

In [15]:
mask_dict = class_distribution_pruning(param_dict,0.1)
pruned_model = apply_masks_to_model(en2de, mask_dict)
torch.save(pruned_model.state_dict(), 'pruned_model_distribution_weights.pt')

In [16]:
!pip install datasets

In [17]:
from datasets import load_dataset

# Load WMT14 en-fr test set
wmt14 = load_dataset("wmt14", "fr-en", split="test")

# Extract English sources and French references
test_sentences_en = [example["translation"]["en"] for example in wmt14]
test_references_fr = [[example["translation"]["fr"]] for example in wmt14]

print(f"Loaded {len(test_sentences_en)} test examples")


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

train-00000-of-00030.parquet:   0%|          | 0.00/252M [00:00<?, ?B/s]

train-00001-of-00030.parquet:   0%|          | 0.00/241M [00:00<?, ?B/s]

train-00002-of-00030.parquet:   0%|          | 0.00/243M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/536k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/40836715 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3003 [00:00<?, ? examples/s]

Loaded 3003 test examples


In [18]:
from sacrebleu import corpus_bleu
import torch

def evaluate_bleu(model, source_sentences, reference_sentences, batch_size=32):
    """
    Evaluate BLEU score with batching for large datasets
    
    Args:
        model: The translation model
        source_sentences: List of source language sentences
        reference_sentences: List of lists of reference translations
        batch_size: Number of sentences to process at once
    
    Returns:
        BLEU score
    """
    translations = []
    
    # Process in batches to avoid memory issues
    for i in range(0, len(source_sentences), batch_size):
        batch = source_sentences[i:i+batch_size]
        batch_translations = model.translate(batch)
        translations.extend(batch_translations)
    
    # Calculate BLEU score
    bleu_score = corpus_bleu(translations, reference_sentences)
    
    return bleu_score.score

In [19]:
# # Evaluate original model
# print("Evaluating original model...")
# original_bleu = evaluate_bleu(en2de, test_sentences_en, test_references_de)
# print(f"Original model BLEU score: {original_bleu:.2f}")

# # Evaluate class-blind pruned model
# print("\nEvaluating class-blind pruned model...")
# pruned_bleu = evaluate_bleu(pruned_model, test_sentences_en, test_references_de)
# print(f"Class-blind pruned model BLEU score: {pruned_bleu:.2f}")

# # Evaluate class-uniform pruned model
# print("\nEvaluating class-uniform pruned model...")
# pruned_blind_bleu = evaluate_bleu(pruned_model_blind, test_sentences_en, test_references_de)
# print(f"Class-uniform pruned model BLEU score: {pruned_blind_bleu:.2f}")

# # Evaluate class-distribution pruned model
# print("\nEvaluating class-distribution pruned model...")
# pruned_distribution_bleu = evaluate_bleu(pruned_model_distribution, test_sentences_en, test_references_de)
# print(f"Class-distribution pruned model BLEU score: {pruned_distribution_bleu:.2f}")


In [ ]:
sparsity_levels = [round(x * 0.1, 1) for x in range(1, 10)]  # 0.1 to 0.9
# sparsity_levels = [0.2]
print(sparsity_levels)
bleu_results = []

with open("bleu_scores.txt", "w") as f:
    for sparsity in sparsity_levels:
        print(f"\nEvaluating at sparsity level: {sparsity}")
        f.write(f"Sparsity Level: {sparsity}\n")

        # You need to define how these pruned models are generated at each sparsity level
        # Assuming you have a function like: prune_model(model, strategy, sparsity)

        # Example placeholders
        model = en2de
        mask_dict = class_distribution_pruning(param_dict,sparsity)
        pruned_model = apply_masks_to_model(model, mask_dict)
      
        bleu_cb = evaluate_bleu(pruned_model, test_sentences_en, test_references_fr)

        result_line = (
            f"Class-Distribution: {bleu_cb:.2f}, "
        )

        print(result_line)
        f.write(result_line)


[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

Evaluating at sparsity level: 0.1
Class-Distribution: 25.10, 

Evaluating at sparsity level: 0.2
Class-Distribution: 17.83, 

Evaluating at sparsity level: 0.3
Class-Distribution: 10.25, 

Evaluating at sparsity level: 0.4
Class-Distribution: 21.36, 

Evaluating at sparsity level: 0.5
Class-Distribution: 5.21, 

Evaluating at sparsity level: 0.6
Class-Distribution: 0.32, 

Evaluating at sparsity level: 0.7
Class-Distribution: 15.97, 

Evaluating at sparsity level: 0.8
Class-Distribution: 12.70, 

Evaluating at sparsity level: 0.9
